In [31]:
# Make sure to run Input_data.ipynb first

# Load all input data
%store -r 

In [32]:
# Make sure to run Input_data.ipynb first

# Load all input data
%store -r 

In [33]:
# Make sure to run Input_data.ipynb first

# Load all input data
%store -r 

In [34]:
# Make sure to run Input_data.ipynb first

# Load all input data
%store -r 

In [35]:
import pyomo.environ as pe
import pyomo.opt as po

In [36]:
# Initialize Model
model = pe.ConcreteModel()

# Sets
model.F = pe.Set(initialize = factories ,ordered = False)
model.T = pe.Set(initialize = transshipment_points ,ordered = False)
model.C = pe.Set(initialize = customers ,ordered = False)
model.M = pe.Set(initialize = modes ,ordered = False)
model.P = pe.Set(initialize = periods, ordered = False)
model.W = pe.Set(initialize = warehouses, ordered = False)

# Decision Variables
# NOTE je kan ook lower en upper bounds geven aan de waarden
model.x = pe.Var(model.F, model.C, model.P, domain = pe.NonNegativeReals)                   # direct tonnes                         !!!TODO : meer uitleg !!!
model.y = pe.Var(model.T, model.C , model.P, domain = pe.NonNegativeReals)                  # indirect tonnes                       !!!TODO : meer uitleg !!!
model.q_bl = pe.Var(model.F, block_trans, model.P, domain = pe.NonNegativeReals)            # quantity block mode                   !!!TODO : meer uitleg !!!  
model.q_s_w = pe.Var(model.F, single_trans, model.P, domain = pe.NonNegativeReals)          # quantity single wagon mode            !!!TODO : meer uitleg !!!
model.q_ss = pe.Var(model.F, shortsea_trans, model.P, domain = pe.NonNegativeReals)         # quantity shortsea ship mode           !!!TODO : meer uitleg !!!
model.q_ba = pe.Var(model.F, barge_trans, model.P, domain = pe.NonNegativeReals)            # quantity barge mode                   !!!TODO : meer uitleg !!!
model.td = pe.Var(model.F, model.C, model.P, domain = pe.NonNegativeIntegers)               # number of trucks via direct route     !!!TODO : meer uitleg !!!
model.tid = pe.Var(model.T, model.C, model.P, domain = pe.NonNegativeIntegers)              # number of trucks via indirect route   !!!TODO : meer uitleg !!!

# #Additional variables 
# model.td_w = pe.Var(model.F, model.W, model.P, domain = pe.NonNegativeIntegers)               
# model.tid_w = pe.Var(model.T, model.W, model.P, domain = pe.NonNegativeIntegers)
# model.x_w = pe.Var(model.F, model.W, model.P, domain = pe.NonNegativeReals)                   
# model.y_w = pe.Var(model.T, model.W, model.P, domain = pe.NonNegativeReals)                  
# model.w_c = pe.Var(model.W, model.C, model.P, domain = pe.NonNegativeReals)
model.inv_level = pe.Var(model.W, model.P, domain = pe.NonNegativeReals)

In [37]:
objExpr = sum(dist_fact_cust[i, j] * transfer_cost * model.x[i,j,p] for i in model.F for j in model.C for p in model.P)     \
        + sum(block_train_cost[i, j] * model.q_bl[i,j,p] for i in model.F for j in block_trans for p in model.P)            \
        + sum(single_wagon_cost[i, j] * model.q_s_w[i,j,p] for i in model.F for j in single_trans for p in model.P)         \
        + sum(shortsea_cost[i, j] * model.q_ss[i,j,p] for i in model.F for j in shortsea_trans for p in model.P)            \
        + sum(barge_cost[i, j] * model.q_ba[i,j,p] for i in model.F for j in barge_trans for p in model.P)                  \
        + sum(dist_point_cust[i, j] * transfer_cost * model.y[i,j,p] for i in model.T for j in model.C for p in model.P)    \
        + sum(handling_costs[i] * model.y[i,j,p] for i in model.T for j in model.C for p in model.P)                        \
        + sum(model.td[i,j,p] * truck_fixed_cost for i in model.F for j in model.C for p in model.P)                        \
        + sum(model.tid[i,j,p] * truck_fixed_cost for i in model.T for j in model.C for p in model.P)                       \
        # + sum(dist_fact_cust[i, j] * transfer_cost * model.x_w[i,j,p] for i in model.F for j in model.W for p in model.P)   \
        # + sum(dist_point_cust[i, j] * transfer_cost * model.y_w[i,j,p] for i in model.T for j in model.W for p in model.P)  \
        # + sum(model.td_w[i,j,p] * truck_fixed_cost for i in model.F for j in model.W for p in model.P)                      \
        # + sum(model.tid_w[i,j,p] * truck_fixed_cost for i in model.T for j in model.W for p in model.P)                       
        
model.obj = pe.Objective(expr = objExpr, sense = pe.minimize)

In [38]:
model.trucksDirect = pe.ConstraintList()
for p in model.P:
    for i in model.F:
        for j in model.C:
            expression = model.td[i,j,p] >= model.x[i,j,p] / truck_cap
            model.trucksDirect.add(expression)

model.trucksIndirect = pe.ConstraintList()
for p in model.P:
    for i in model.T:
        for j in model.C:
            expression = model.tid[i,j,p] >= model.y[i,j,p] / truck_cap
            model.trucksIndirect.add(expression)

# model.trucksDirectW = pe.ConstraintList()
# for p in model.P:
#     for i in model.F:
#         for j in model.W:
#             expression = model.td_w[i,j,p] >= model.x_w[i,j,p] / truck_cap
#             model.trucksDirectW.add(expression)

# model.trucksIndirectW = pe.ConstraintList()
# for p in model.P:
#     for i in model.T:
#         for j in model.W:
#             expression = model.tid_w[i,j,p] >= model.y_w[i,j,p] / truck_cap
#             model.trucksIndirectW.add(expression)

#WITH W AS WAREHOUSE
# model.prodCap = pe.ConstraintList()
# for p in model.P:
#     for i in model.F:
#         expression = sum(model.x[i,j,p] for j in model.C) + sum(model.x_w[i,j,p] for j in model.W) + sum(model.q_bl[i,j,p] for j in block_trans) + sum(model.q_s_w[i,j,p] for j in single_trans) + sum(model.q_ss[i,j,p] for j in shortsea_trans) + sum(model.q_ba[i,j,p] for j in barge_trans) <= production_cap[i]
#         model.prodCap.add(expression)

# model.meetDemand = pe.ConstraintList()
# for p in model.P:
#     for j in model.C:
#         expression = sum(model.x[i,j,p] for i in model.F) + sum(model.y[i,j,p] for i in model.T) + sum(model.w_c[i,j,p] for i in model.W if i == j) >= demands[p][j]
#         model.meetDemand.add(expression)

model.prodCap = pe.ConstraintList()
for p in model.P:
    for i in model.F:
        expression = sum(model.x[i,j,p] for j in model.C) + sum(model.q_bl[i,j,p] for j in block_trans) + sum(model.q_s_w[i,j,p] for j in single_trans) + sum(model.q_ss[i,j,p] for j in shortsea_trans) + sum(model.q_ba[i,j,p] for j in barge_trans) <= production_cap[i]
        model.prodCap.add(expression)

model.meetDemand = pe.ConstraintList()
for p in model.P:
    for j in model.C:
        expression = sum(model.x[i,j,p] for i in model.F) + sum(model.y[i,j,p] for i in model.T) >= demands[p][j] - model.inv_level[j,p]
        model.meetDemand.add(expression)

# model.noDelivery_w_c = pe.ConstraintList()
# for p in model.P:
#     for i in model.W:
#         for j in model.C: 
#             if i != j: 
#                 expression = model.w_c[i,j,p] == 0
#                 model.noDelivery_w_c.add(expression)

# model.flowBalance = pe.ConstraintList()
# for p in model.P:
#     for j in block_trans:
#         expression_bl = sum(model.q_bl[i,j,p] for i in model.F) 
#         expression_s_w = sum(model.q_s_w[i,j,p] for i in model.F)
#     for j in shortsea_trans:
#         expression_ss = sum(model.q_ss[i,j,p] for i in model.F) 
#     for j in barge_trans:
#         expression_ba = sum(model.q_ba[i,j,p] for i in model.F) 
#     for j in model.T: 
#         expression = expression_bl + expression_s_w + expression_ss + expression_ba == sum(model.y[j,k,p] for k in model.C) + sum(model.y_w[j,w,p] for w in model.W)
#         model.flowBalance.add(expression)

model.flowBalance = pe.ConstraintList()
for p in model.P:
    total_quantity_trans = {"Robert Schmidtz": 0, "Wiechers": 0, "Haeger und Schmidt": 0, "Neska/UCT Neuss": 0,
			"WTA": 0, "Siefert spedition": 0, "Rhenus logistics": 0}
    for j in block_trans:
        expression_bl = sum(model.q_bl[i,j,p] for i in model.F)
        expression_s_w = sum(model.q_s_w[i,j,p] for i in model.F) 
        total_train = expression_bl + expression_s_w
        total_quantity_trans[j] += total_train
    for j in shortsea_trans:
        expression_ss = sum(model.q_ss[i,j,p] for i in model.F) 
        total_quantity_trans[j] += expression_ss
    for j in barge_trans:
        expression_ba = sum(model.q_ba[i,j,p] for i in model.F) 
        total_quantity_trans[j] += expression_ba
    for j in model.T: 
        expression = total_quantity_trans[j] == sum(model.y[j,k,p] for k in model.C)
        model.flowBalance.add(expression)

model.inventory = pe.ConstraintList()
for w in model.W:
    expression = model.inv_level[w,1] == InvLev[w] + sum(model.x[i,w,1] for i in model.F) + sum(model.y[i,w,1] for i in model.T) - demands[1][w] 
    model.inventory.add(expression)  

model.update_inventory = pe.ConstraintList()
for p in model.P:
    for w in model.W: 
        if p > 1:
            expression = model.inv_level[w,p] == model.inv_level[w,p-1] + sum(model.x[i,w,p] for i in model.F) + sum(model.y[i,w,p] for i in model.T) - demands[p][w]
            model.update_inventory.add(expression)

model.inventory_cap = pe.ConstraintList()
for p in model.P:
    for w in model.W:
        expression = model.inv_level[w,p] <= InvCap[w]
        model.inventory_cap.add(expression)

In [39]:
solver = po.SolverFactory('gurobi')
result = solver.solve(model, tee = True, timelimit=360) # timelimit werkt wel maar vgm blijft ie op de achtergrond runnen


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2022-03-21
Using license file /Users/tessavanliempt/gurobi.lic
Read LP format model from file /var/folders/qp/3z7wqsnn5xn47x6dpmtpq_vh0000gp/T/tmp0y8bnqg9.pyomo.lp
Reading time = 0.07 seconds
x1951: 1141 rows, 1951 columns, 4773 nonzeros
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1141 rows, 1951 columns and 4773 nonzeros
Model fingerprint: 0x5c9b2f9d
Variable types: 1151 continuous, 800 integer (0 binary)
Coefficient statistics:
  Matrix range     [4e-02, 1e+00]
  Objective range  [2e+00, 1e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 6e+03]
Found heuristic solution: objective 2.368479e+07
Presolve removed 89 rows and 61 columns
Presolve time: 0.03s
Presolved: 1052 rows, 1890 columns, 4492 nonzeros
Found heuris

In [40]:
print("Objective value = "+ str(pe.value(model.obj)))

Objective value = 334238.5091999937
